## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lavrentiya,RU,65.5833,-171.0000,-16.26,80,0,5.70,clear sky
1,1,Qaanaaq,GL,77.4840,-69.3632,6.49,78,1,5.97,clear sky
2,2,Saskylakh,RU,71.9167,114.0833,-15.93,84,95,20.56,overcast clouds
3,3,San Quintin,MX,30.4833,-115.9500,59.54,60,21,15.97,few clouds
4,4,Vardo,NO,70.3705,31.1107,14.00,67,20,27.63,snow


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?70
What is the maximum temperature you would like for your trip?90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Saint-Philippe,RE,-21.3585,55.7679,77.00,69,0,5.75,light rain
6,6,Rikitea,PF,-23.1203,-134.9692,78.82,75,10,17.43,clear sky
7,7,Mar Del Plata,AR,-38.0023,-57.5575,73.40,68,0,9.22,clear sky
8,8,Kapaa,US,22.0752,-159.3190,75.20,83,40,5.75,light rain
12,12,Bambous Virieux,MU,-20.3428,57.7575,78.80,83,75,10.36,broken clouds
13,13,Sinnamary,GF,5.3833,-52.9500,77.81,85,92,13.27,overcast clouds
15,15,Mancora,PE,-4.1078,-81.0475,84.20,65,40,10.36,light rain
23,23,Blackwater,AU,-23.5833,148.8833,79.11,66,100,7.81,overcast clouds
25,25,Padang,ID,-0.9492,100.3543,73.40,94,40,2.30,scattered clouds
26,26,Atuona,PF,-9.8000,-139.0333,79.93,75,26,15.50,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()
preferred_cities_df.count()

# No nulls; do not need to use 'dropna()'

City_ID                271
City                   271
Country                271
Lat                    271
Lng                    271
Max Temp               271
Humidity               271
Cloudiness             271
Wind Speed             271
Current Description    271
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Saint-Philippe,RE,77.00,light rain,-21.3585,55.7679,
6,Rikitea,PF,78.82,clear sky,-23.1203,-134.9692,
7,Mar Del Plata,AR,73.40,clear sky,-38.0023,-57.5575,
8,Kapaa,US,75.20,light rain,22.0752,-159.3190,
12,Bambous Virieux,MU,78.80,broken clouds,-20.3428,57.7575,
13,Sinnamary,GF,77.81,overcast clouds,5.3833,-52.9500,
15,Mancora,PE,84.20,light rain,-4.1078,-81.0475,
23,Blackwater,AU,79.11,overcast clouds,-23.5833,148.8833,
25,Padang,ID,73.40,scattered clouds,-0.9492,100.3543,
26,Atuona,PF,79.93,scattered clouds,-9.8000,-139.0333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
   
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
     
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Saint-Philippe,RE,77.00,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
6,Rikitea,PF,78.82,clear sky,-23.1203,-134.9692,Pension Maro'i
7,Mar Del Plata,AR,73.40,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
8,Kapaa,US,75.20,light rain,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
12,Bambous Virieux,MU,78.80,broken clouds,-20.3428,57.7575,Casa Tia Villa
13,Sinnamary,GF,77.81,overcast clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
15,Mancora,PE,84.20,light rain,-4.1078,-81.0475,Máncora Marina Hotel
23,Blackwater,AU,79.11,overcast clouds,-23.5833,148.8833,Blackwater Motor Inn
25,Padang,ID,73.40,scattered clouds,-0.9492,100.3543,Mercure Padang
26,Atuona,PF,79.93,scattered clouds,-9.8000,-139.0333,Villa Enata


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)


info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))